<a href="https://colab.research.google.com/github/adi-kiran/LightGCN-Simplified/blob/main/code/LightGCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
import copy
import gzip
import json

from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics, preprocessing

import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.typing import Adj
from torch_geometric.utils import degree
# from torch_geometric.utils import structured_negative_sampling
from torch_sparse import SparseTensor, matmul

2.1.0+cu118
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# dataset URLs
poetryBooksUrl = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/goodreads_books_poetry.json.gz"
poetryInteractionsUrl = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/goodreads_interactions_poetry.json.gz"
poetryReviewsUrl = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/goodreads_reviews_poetry.json.gz"
# dataset download base dir path in mounted Drive
dataPath = "/content/drive/MyDrive/MLG/Project/BookRecommendationSystem/dataset/."
# dataset file paths
poetryBooksJsonFilePath = "/".join([dataPath,poetryBooksUrl.split("/")[-1]])
poetryInteractionsJsonFilePath = "/".join([dataPath,poetryInteractionsUrl.split("/")[-1]])
poetryReviewsJsonFilePath = "/".join([dataPath,poetryReviewsUrl.split("/")[-1]])

In [ ]:
reviewsDf = pd.read_json(poetryReviewsJsonFilePath,lines=True)

In [ ]:
reviewsDf.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,3ca7375dba942a760e53b726c472a7dd,402128,28423ff309bc896c071a8d9df4a10e8a,5,I have three younger siblings and we grew up w...,Tue Jun 12 08:59:04 -0700 2012,Fri Jun 15 11:41:12 -0700 2012,,,0,0
1,0ef32090550901ead25cb0ea21c4d36b,92270,2db1180992e2b0b1631a3ac5644bde84,5,This is my favorite collection of poetry.,Mon Apr 14 18:42:40 -0700 2014,Mon Apr 14 18:43:05 -0700 2014,Wed Jan 01 00:00:00 -0800 1997,,0,0
2,0ef32090550901ead25cb0ea21c4d36b,908708,bca57fa40e92c9261b00b03dbebd96fe,4,"He's so disturbing. So very, very disturbing.",Tue Apr 22 13:58:10 -0700 2008,Tue Apr 22 13:58:33 -0700 2008,,,0,0
3,d37b46b2190ed7c518259f29b47a9b36,253264,cb1ebc02d8b2aff15735d513877463ce,5,I just reread this play for a class I am takin...,Wed Sep 27 19:08:08 -0700 2017,Sat Sep 30 06:39:45 -0700 2017,Wed Sep 27 00:00:00 -0700 2017,Tue Sep 26 00:00:00 -0700 2017,1,0
4,af157d0205b8a901dee6d4a2aed7e6ad,70885,8dca128b8e869048a7442c18659dbece,5,"Cuanto mas leo, mas me gusta. Su poesia es env...",Thu Jun 18 20:00:03 -0700 2015,Thu Jun 18 20:01:29 -0700 2015,Thu Jun 18 00:00:00 -0700 2015,Tue Jun 16 00:00:00 -0700 2015,0,0


In [ ]:
reviewsDf.info()
reviewsDf.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154555 entries, 0 to 154554
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       154555 non-null  object
 1   book_id       154555 non-null  int64 
 2   review_id     154555 non-null  object
 3   rating        154555 non-null  int64 
 4   review_text   154555 non-null  object
 5   date_added    154555 non-null  object
 6   date_updated  154555 non-null  object
 7   read_at       154555 non-null  object
 8   started_at    154555 non-null  object
 9   n_votes       154555 non-null  int64 
 10  n_comments    154555 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 13.0+ MB


,book_id,rating,n_votes,n_comments
count,1.545550e+05,154555.000000,154555.000000,154555.000000
mean,1.023782e+07,3.815205,1.525632,0.252557
std,1.026113e+07,1.310501,7.232086,1.638360
min,2.340000e+02,0.000000,-1.000000,-1.000000
25%,5.226630e+05,3.000000,0.000000,0.000000
50%,6.928895e+06,4.000000,0.000000,0.000000
75%,1.822272e+07,5.000000,1.000000,0.000000
max,3.648548e+07,5.000000,1065.000000,168.000000


In [ ]:
reviewsDf.rating.value_counts()

5    58510
4    47266
3    28028
2     9942
0     6732
1     4077
Name: rating, dtype: int64

In [ ]:
# Nodes
#   1. Users - node features?
#   2. Books - node features?
#       Possible Node features
#           1. Average_rating
#           2. is_ebook
#           3. title_without_series
#           4. ratings_count
#           5. text_reviews_count
#           6. language_code
# Edges
#   1. User--->Book - YES - (user_id -> book_id) - Directed Edge??
#           - Possible Edge Features:
#               1.Rating
#               2.Review Text (Convert to embedding? BERT, Word2Vec, etc)
#               3.Number of Votes ? Most will likely have 0 Votes.
#               4.Number of Comments ? Most will have 0 comments. Also comments are generally biased. The commenter will either agree wholly or critise the review. How do we characterize this? Do we even need to?
#   2. Book--->Book - MAYBE? - (book_id -> book_id) - Undirected Edge - Book data has a field called similar books
#            - Possible Edge Features:
#               1.Maybe a score of 1 for same genre/category and a score of 5 for books marked as similar in the dataset?
#   3. User--->User - NO - No data for user-user relations

In [ ]:
# For testing
#     I'm going to assume that we have only user->book edges and the edge weight is rating
#     Going to only use the reviewDf and consider only the user_id, book_id and ratings. Dropping all other columns

In [ ]:
rating_df = reviewsDf[["book_id","user_id","rating"]]
rating_df.head()

,book_id,user_id,rating
0,402128,3ca7375dba942a760e53b726c472a7dd,5
1,92270,0ef32090550901ead25cb0ea21c4d36b,5
2,908708,0ef32090550901ead25cb0ea21c4d36b,4
3,253264,d37b46b2190ed7c518259f29b47a9b36,5
4,70885,af157d0205b8a901dee6d4a2aed7e6ad,5


In [ ]:
print('%s users'%(len(rating_df['user_id'].unique())))
print('%s books'%(len(rating_df['book_id'].unique())))
print("%s ratings"%(len(rating_df)))

47400 users
36412 books
154555 ratings


In [ ]:
# perform encoding preprocessing to ensure that user_id and book_id are both in the range of [0, unique_count] so it won't cause out of bound issue when indexing embeddings
lbl_encoder_user = preprocessing.LabelEncoder()
lbl_encoder_book = preprocessing.LabelEncoder()

rating_df["user_id_idx"] = lbl_encoder_user.fit_transform(rating_df["user_id"].values)
rating_df["book_id_idx"] = lbl_encoder_book.fit_transform(rating_df["book_id"].values)

<ipython-input-12-b7eccea1a9e8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_df["user_id_idx"] = lbl_encoder_user.fit_transform(rating_df["user_id"].values)
<ipython-input-12-b7eccea1a9e8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_df["book_id_idx"] = lbl_encoder_book.fit_transform(rating_df["book_id"].values)


In [ ]:
print("User IDs now range from 0 to %s"%(rating_df["user_id_idx"].max()))
print("Book IDs now range from 0 to %s"%(rating_df["book_id_idx"].max()))
print("\nRatings are distributed as :")
print(rating_df.rating.value_counts())

User IDs now range from 0 to 47399
Book IDs now range from 0 to 36411

Ratings are distributed as :
5    58510
4    47266
3    28028
2     9942
0     6732
1     4077
Name: rating, dtype: int64


In [ ]:
# load edges between users and books
def load_edge_dataframe(df,
                  src_index_col,
                  dst_index_col,
                  rating_col,
                  rating_threshold=3):
    """Converts datafrane containing edges between users and items into an edge matrix
    Args:
        df (dataframe): dataframe containing the users ratings for books
        src_index_col (str): column name of user ids
        dst_index_col (str): column name of book ids
        rating_col (str): column name of user-book rating
        rating_threshold (int, optional): Threshold to determine existance of edge. If rating less than threshold value, edge dropped
    Returns:
        tensor: 2 by N matrix containing the node ids of N user-item edges
        (N here is the number of interactions)
    """

    edge_index = None

    # Constructing COO format edge_index from input rating events

    # get user_ids from rating events in the order of occurance
    src = [user_id for user_id in df[src_index_col]]
    # get book_id from rating events in the order of occurance
    dst = [item_id for item_id in df[dst_index_col]]

    # apply rating threshold
    edge_attr = torch.from_numpy(df[rating_col].values).view(-1, 1).to(torch.long) >= rating_threshold

    edge_index = [[], []]
    for i in range(edge_attr.shape[0]):
        if edge_attr[i]:
            edge_index[0].append(src[i])
            edge_index[1].append(dst[i])
    return torch.tensor(edge_index)

In [ ]:
edge_index = load_edge_dataframe(
    rating_df,
    src_index_col='user_id_idx',
    dst_index_col='book_id_idx',
    rating_col='rating',
    rating_threshold=3,
)

In [ ]:
# splitting the edges of the graph using a 60/20/20 train/validation/test split
num_users, num_books = len(lbl_encoder_user.classes_), len(lbl_encoder_book.classes_)
num_interactions = edge_index.shape[1]

print(f"Number of users : {num_users}")
print(f"Number of books : {num_books}")
print(f"Number of interactions : {num_interactions}")

all_edges = [i for i in range(num_interactions)]

train_edges, test_edges = train_test_split(all_edges, test_size=0.4, random_state=1)
val_edges, test_edges = train_test_split(test_edges, test_size=0.5, random_state=1)
print(train_edges)

train_edge_index = edge_index[:, train_edges]
val_edge_index = edge_index[:, val_edges]
test_edge_index = edge_index[:, test_edges]

Number of users : 47400
Number of books : 36412
Number of interactions : 133804
[24285, 82036, 133682, 43081, 92022, 95987, 27864, 21952, 7065, 19230, 35417, 34385, 45822, 115371, 113120, 106151, 46453, 13722, 89160, 38284, 113555, 83026, 56987, 39967, 8673, 82972, 14793, 35688, 121969, 8239, 130335, 44353, 37681, 55769, 77152, 4968, 55249, 99797, 59322, 131685, 62585, 122944, 20996, 121933, 31994, 50395, 113204, 101585, 65400, 44216, 55748, 23696, 20930, 815, 62765, 128236, 74411, 87255, 34324, 86889, 106962, 9270, 124416, 88170, 55411, 35231, 73320, 47878, 119147, 110558, 89766, 38646, 98080, 33086, 85789, 21752, 75973, 74922, 72173, 15407, 110263, 109685, 53728, 42974, 92191, 113238, 41319, 85508, 59211, 3795, 34889, 50488, 74984, 1300, 42264, 43972, 31086, 12834, 6627, 70425, 37260, 9684, 37042, 98869, 90897, 109928, 5189, 237, 63706, 4325, 73231, 86417, 49719, 40487, 90270, 120218, 50705, 24528, 60854, 8002, 88129, 113904, 83417, 124304, 123291, 62391, 109663, 33919, 103652, 22917

In [ ]:
# convert edge indices into Sparse Tensors: https://pytorch-geometric.readthedocs.io/en/latest/notes/sparse_tensor.html
train_sparse_edge_index = SparseTensor(
                            row=train_edge_index[0],
                            col=train_edge_index[1],
                            sparse_sizes=(num_users + num_books, num_users + num_books)
                        )
val_sparse_edge_index = SparseTensor(
                            row=val_edge_index[0],
                            col=val_edge_index[1],
                            sparse_sizes=(num_users + num_books, num_users + num_books)
                        )
test_sparse_edge_index = SparseTensor(
                            row=test_edge_index[0],
                            col=test_edge_index[1],
                            sparse_sizes=(num_users + num_books, num_users + num_books)
                        )

In [ ]:
from torch_geometric.utils import coalesce, degree, remove_self_loops
from torch_geometric.utils.num_nodes import maybe_num_nodes

def structured_negative_sampling(edge_index, num_nodes, contains_neg_self_loops = False):
    r"""Samples a negative edge :obj:`(i,k)` for every positive edge
    :obj:`(i,j)` in the graph given by :attr:`edge_index`, and returns it as a
    tuple of the form :obj:`(i,j,k)`.

    Args:
        edge_index (LongTensor): The edge indices.
        num_nodes (int or Tuple[int, int], optional): The number of nodes,
            *i.e.* :obj:`max_val + 1` of :attr:`edge_index`.
            If given as a tuple, then :obj:`edge_index` is interpreted as a
            bipartite graph with shape :obj:`(num_src_nodes, num_dst_nodes)`.
            (default: :obj:`None`)
        contains_neg_self_loops (bool, optional): If set to
            :obj:`False`, sampled negative edges will not contain self loops.
            (default: :obj:`True`)

    :rtype: (LongTensor, LongTensor, LongTensor)

    Example:

        >>> edge_index = torch.as_tensor([[0, 0, 1, 2],
        ...                               [0, 1, 2, 3]])
        >>> structured_negative_sampling(edge_index)
        (tensor([0, 0, 1, 2]), tensor([0, 1, 2, 3]), tensor([2, 3, 0, 2]))

    """
    if not structured_negative_sampling_feasible(
            edge_index=edge_index, num_nodes=num_nodes,
            contains_neg_self_loops=contains_neg_self_loops):
        raise Exception("structured_negative_sampling is not feasible")

    size = num_nodes
    bipartite = isinstance(size, (tuple, list))
    size = maybe_num_nodes(edge_index) if size is None else size
    contains_neg_self_loops = True if bipartite else contains_neg_self_loops
    num_nodes = size[1] if bipartite else size

    row, col = edge_index.cpu()
    pos_idx = row * num_nodes + col
    if not contains_neg_self_loops:
        loop_idx = torch.arange(num_nodes) * (num_nodes + 1)
        pos_idx = torch.cat([pos_idx, loop_idx], dim=0)

    rand = torch.randint(num_nodes, (row.size(0), ), dtype=torch.long)
    neg_idx = row * num_nodes + rand

    mask = torch.from_numpy(np.isin(neg_idx, pos_idx)).to(torch.bool)
    rest = mask.nonzero(as_tuple=False).view(-1)
    while rest.numel() > 0:  # pragma: no cover
        tmp = torch.randint(num_nodes, (rest.size(0), ), dtype=torch.long)
        rand[rest] = tmp
        neg_idx = row[rest] * num_nodes + tmp

        mask = torch.from_numpy(np.isin(neg_idx, pos_idx)).to(torch.bool)
        rest = rest[mask]

    return edge_index[0], edge_index[1], rand.to(edge_index.device)


def structured_negative_sampling_feasible(edge_index,num_nodes = None,contains_neg_self_loops = False):
    r"""Returns :obj:`True` if
    :meth:`~torch_geometric.utils.structured_negative_sampling` is feasible
    on the graph given by :obj:`edge_index`.
    :meth:`~torch_geometric.utils.structured_negative_sampling` is infeasible
    if atleast one node is connected to all other nodes.

    Args:
        edge_index (LongTensor): The edge indices.
        num_nodes (int or Tuple[int, int], optional): The number of nodes,
            *i.e.* :obj:`max_val + 1` of :attr:`edge_index`.
            If given as a tuple, then :obj:`edge_index` is interpreted as a
            bipartite graph with shape :obj:`(num_src_nodes, num_dst_nodes)`.
            (default: :obj:`None`)
        contains_neg_self_loops (bool, optional): If set to
            :obj:`False`, sampled negative edges will not contain self loops.
            (default: :obj:`True`)

    :rtype: bool

    Examples:

        >>> edge_index = torch.LongTensor([[0, 0, 1, 1, 2, 2, 2],
        ...                                [1, 2, 0, 2, 0, 1, 1]])
        >>> structured_negative_sampling_feasible(edge_index, 3, False)
        False

        >>> structured_negative_sampling_feasible(edge_index, 3, True)
        True
    """
    size = num_nodes
    bipartite = isinstance(size, (tuple, list))
    size = maybe_num_nodes(edge_index) if size is None else size
    contains_neg_self_loops = True if bipartite else contains_neg_self_loops
    num_nodes = size if not bipartite else size[0]
    max_num_neighbors = num_nodes if not bipartite else size[1]

    edge_index = coalesce(edge_index, num_nodes=num_nodes)

    if not contains_neg_self_loops:
        edge_index, _ = remove_self_loops(edge_index)
        max_num_neighbors -= 1  # Reduce number of valid neighbors

    deg = degree(edge_index[0], num_nodes)
    # True if there exists no node that is connected to all other nodes.
    return bool(torch.all(deg < max_num_neighbors))

In [ ]:
# function which random samples a mini-batch of positive and negative samples
def sample_mini_batch(batch_size, edge_index):
    """Randomly samples indices of a minibatch given an adjacency matrix

    Args:
        batch_size (int): minibatch size
        edge_index (torch.Tensor): 2 by N list of edges

    Returns:
        tuple: user indices, positive item indices, negative item indices
    """
    edges = structured_negative_sampling(edge_index,[num_users,num_books])
    edges = torch.stack(edges, dim=0)
    indices = random.choices([i for i in range(edges[0].shape[0])], k=batch_size)
    batch = edges[:, indices]
    user_indices, pos_item_indices, neg_item_indices = batch[0], batch[1], batch[2]
    return user_indices, pos_item_indices, neg_item_indices

In [ ]:
# LightGCN model
class LightGCN(MessagePassing):

    def __init__(self, num_users, num_items, embedding_dim=64, num_layers=2):
        """Initialize LightGCN Model
        Args:
            num_users (int): Number of users
            num_items (int): Number of items
            embedding_dim (int): Dimensionality of embeddings. Defaults to 64.
            num_layers (int): Number of message passing layers. Defaults to 2.
        """
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        # The user and item embeddings are the model parameters that will be learnt
        # Initialise them with normalised weights
        self.users_emb = torch.nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.embedding_dim) # User embedding for layer 0, i.e., e_u^0
        self.items_emb = torch.nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.embedding_dim) # Item embedding for layer 0, i.e., e_i^0
        torch.nn.init.normal_(self.users_emb.weight, std=0.1)
        torch.nn.init.normal_(self.items_emb.weight, std=0.1)

    def forward(self, edge_index: SparseTensor):
        """Forward propagation of LightGCN Model.
        Args:
            edge_index (SparseTensor): list of edges in the format [[usr_ind_1, usr_ind_2,......,usr_ind_n],[itm_ind_1,itm_ind_2,....,itm_ind_n]], where (usr_ind_i,itm_ind_i) correspond to an edge
        Returns:
            tuple (Tensor): e_u_k, e_u_0, e_i_k, e_i_0 where e_u_k  = Final layer user embedding, e_u_0 = Initial user embedding, e_i_k = Final layer item embedding, e_i_0 = Initial item embedding
        """
        # compute the normalized adjacency matrix
        edge_index_norm = gcn_norm(edge_index, add_self_loops=False) # no self loops considered for LightGCN
        # combine the user and item embeddings to get the initial (0th layer) embedding
        emb_0 = torch.cat([self.users_emb.weight, self.items_emb.weight]) # E^0 = [e_u^K, e_i^K]
        embs = [emb_0]
        emb_k = emb_0
        # calculate the embedding after each layer
        for i in range(self.num_layers):
            emb_k = self.propagate(edge_index_norm, x=emb_k)
            embs.append(emb_k)
        # get the final layer embedding
        embs = torch.stack(embs, dim=1)
        emb_final = torch.mean(embs, dim=1) # E^K
        # split this final layer embedding into the final layer user embedding and final layer item embedding (e_u^K and e_i^K)
        users_emb_final, items_emb_final = torch.split(emb_final, [self.num_users, self.num_items])
        # return the final and initial user and item embeddings in the order, e_u^K, e_u^0, e_i^K, e_i^0
        return users_emb_final, self.users_emb.weight, items_emb_final, self.items_emb.weight

    def message(self, x_j):
        return x_j

    def message_and_aggregate(self, adj_t, x):
        return matmul(adj_t, x)

In [ ]:
def bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, lambda_val):
    """Bayesian Personalized Ranking Loss as described in https://arxiv.org/abs/1205.2618
    Args:
        users_emb_final (torch.Tensor): e_u_k
        users_emb_0 (torch.Tensor): e_u_0
        pos_items_emb_final (torch.Tensor): positive e_i_k
        pos_items_emb_0 (torch.Tensor): positive e_i_0
        neg_items_emb_final (torch.Tensor): negative e_i_k
        neg_items_emb_0 (torch.Tensor): negative e_i_0
        lambda_val (float): lambda value for scaling the regularization loss term
    Returns:
        torch.Tensor: scalar bpr loss value
    """
    reg_loss = lambda_val * (users_emb_0.norm(2).pow(2) + pos_items_emb_0.norm(2).pow(2) + neg_items_emb_0.norm(2).pow(2)) # L2 loss * lambda
    # TO DO: I think L2 is being calculated incorrectly here. I think it should be multiplied by 0.5 . Confirm the formula and then change it.
    # reg_loss = lambda_val * 0.5 * (users_emb_0.norm(2).pow(2) + pos_items_emb_0.norm(2).pow(2) + neg_items_emb_0.norm(2).pow(2)) # L2 loss * lambda

    pos_scores = torch.mul(users_emb_final, pos_items_emb_final) # (e_u * e_i_pos)
    pos_scores = torch.sum(pos_scores, dim=-1) # predicted scores of positive samples
    neg_scores = torch.mul(users_emb_final, neg_items_emb_final) # (e_u * e_i_neg)
    neg_scores = torch.sum(neg_scores, dim=-1) # predicted scores of negative samples
    loss = -torch.mean(torch.nn.functional.softplus(pos_scores - neg_scores)) + reg_loss
    # TO DO: See if the sigmoid is better than the softplus
    # loss = -F.logsigmoid(pos_scores - neg_scores).sum() + reg_loss
    return loss

In [ ]:
# helper function to get N_u
def get_user_positive_items(edge_index):
    """Generates dictionary of positive items for each user

    Args:
        edge_index (torch.Tensor): 2 by N list of edges

    Returns:
        dict: dictionary of positive items for each user
    """
    user_pos_items = {}
    for i in range(edge_index.shape[1]):
        user = edge_index[0][i].item()
        item = edge_index[1][i].item()
        if user not in user_pos_items:
            user_pos_items[user] = []
        user_pos_items[user].append(item)
    return user_pos_items

In [ ]:
# computes recall@K and precision@K
def RecallPrecision_ATk(groundTruth, r, k):
    """Computers recall @ k and precision @ k

    Args:
        groundTruth (list): list of lists containing highly rated items of each user
        r (list): list of lists indicating whether each top k item recommended to each user
            is a top k ground truth item or not
        k (intg): determines the top k items to compute precision and recall on

    Returns:
        tuple: recall @ k, precision @ k
    """
    num_correct_pred = torch.sum(r, dim=-1)  # number of correctly predicted items per user
    # number of items liked by each user in the test set
    user_num_liked = torch.Tensor([len(groundTruth[i])
                                  for i in range(len(groundTruth))])
    recall = torch.mean(num_correct_pred / user_num_liked)
    precision = torch.mean(num_correct_pred) / k
    return recall.item(), precision.item()

In [ ]:
# computes NDCG@K
def NDCGatK_r(groundTruth, r, k):
    """Computes Normalized Discounted Cumulative Gain (NDCG) @ k

    Args:
        groundTruth (list): list of lists containing highly rated items of each user
        r (list): list of lists indicating whether each top k item recommended to each user
            is a top k ground truth item or not
        k (int): determines the top k items to compute ndcg on

    Returns:
        float: ndcg @ k
    """
    assert len(r) == len(groundTruth)

    test_matrix = torch.zeros((len(r), k))

    for i, items in enumerate(groundTruth):
        length = min(len(items), k)
        test_matrix[i, :length] = 1
    max_r = test_matrix
    idcg = torch.sum(max_r * 1. / torch.log2(torch.arange(2, k + 2)), axis=1)
    dcg = r * (1. / torch.log2(torch.arange(2, k + 2)))
    dcg = torch.sum(dcg, axis=1)
    idcg[idcg == 0.] = 1.
    ndcg = dcg / idcg
    ndcg[torch.isnan(ndcg)] = 0.
    return torch.mean(ndcg).item()

In [ ]:
# wrapper function to get evaluation metrics
def get_metrics(users_emb_final, items_emb_final, edge_index, exclude_edge_indices, k):
    """Computes the evaluation metrics: recall, precision, and ndcg @ k

    Args:
        model (LighGCN): lightgcn model
        edge_index (torch.Tensor): 2 by N list of edges for split to evaluate
        exclude_edge_indices ([type]): 2 by N list of edges for split to discount from evaluation
        k (int): determines the top k items to compute metrics on

    Returns:
        tuple: recall @ k, precision @ k, ndcg @ k
    """

    # get ratings between every user and item - shape is num users x num movies
    rating = torch.matmul(users_emb_final, items_emb_final.T)

    for exclude_edge_index in exclude_edge_indices:
        # gets all the positive items for each user from the edge index
        user_pos_items = get_user_positive_items(exclude_edge_index)
        # get coordinates of all edges to exclude
        exclude_users = []
        exclude_items = []
        for user, items in user_pos_items.items():
            exclude_users.extend([user] * len(items))
            exclude_items.extend(items)

        # set ratings of excluded edges to large negative value
        rating[exclude_users, exclude_items] = -(1 << 10)

    # get the top k recommended items for each user
    _, top_K_items = torch.topk(rating, k=k)

    # get all unique users in evaluated split
    users = edge_index[0].unique()

    test_user_pos_items = get_user_positive_items(edge_index)

    # convert test user pos items dictionary into a list
    test_user_pos_items_list = [
        test_user_pos_items[user.item()] for user in users]

    # determine the correctness of topk predictions
    r = []
    for user in users:
        ground_truth_items = test_user_pos_items[user.item()]
        label = list(map(lambda x: x in ground_truth_items, top_K_items[user]))
        r.append(label)
    r = torch.Tensor(np.array(r).astype('float'))

    recall, precision = RecallPrecision_ATk(test_user_pos_items_list, r, k)
    ndcg = NDCGatK_r(test_user_pos_items_list, r, k)

    return recall, precision, ndcg

In [ ]:
# wrapper function to evaluate model
def evaluation(model, edge_index, sparse_edge_index, exclude_edge_indices, k, lambda_val):
    """Evaluates model loss and metrics including recall, precision, ndcg @ k

    Args:
        model (LighGCN): lightgcn model
        edge_index (torch.Tensor): 2 by N list of edges for split to evaluate
        sparse_edge_index (sparseTensor): sparse adjacency matrix for split to evaluate
        exclude_edge_indices ([type]): 2 by N list of edges for split to discount from evaluation
        k (int): determines the top k items to compute metrics on
        lambda_val (float): determines lambda for bpr loss

    Returns:
        tuple: bpr loss, recall @ k, precision @ k, ndcg @ k
    """
    # get embeddings
    users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(sparse_edge_index)
    recall, precision, ndcg = get_metrics(users_emb_final, items_emb_final, edge_index, exclude_edge_indices, k)
    edges = structured_negative_sampling(edge_index, [num_users,num_books], contains_neg_self_loops=False)
    user_indices, pos_item_indices, neg_item_indices = edges[0], edges[1], edges[2]
    users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]
    pos_items_emb_final, pos_items_emb_0 = items_emb_final[pos_item_indices], items_emb_0[pos_item_indices]
    neg_items_emb_final, neg_items_emb_0 = items_emb_final[neg_item_indices], items_emb_0[neg_item_indices]

    loss = bpr_loss(users_emb_final,
                    users_emb_0,
                    pos_items_emb_final,
                    pos_items_emb_0,
                    neg_items_emb_final,
                    neg_items_emb_0,
                    lambda_val).item()

    return loss, recall, precision, ndcg

In [ ]:
# define contants
# ITERATIONS = 10000
# BATCH_SIZE = 1024
# LR = 1e-3
# ITERS_PER_EVAL = 200
# ITERS_PER_LR_DECAY = 200
# K = 20
# LAMBDA = 1e-4

In [ ]:
# define contants
ITERATIONS = 3500
BATCH_SIZE = 1024
LR = 1e-3
ITERS_PER_EVAL = 200
ITERS_PER_LR_DECAY = 200
K = 20
LAMBDA = 1e-8

In [ ]:
# # define contants
# ITERATIONS = 10000
# BATCH_SIZE = 1024
# LR = 1e-3
# ITERS_PER_EVAL = 200
# ITERS_PER_LR_DECAY = 200
# K = 20
# LAMBDA = 1e-7

In [ ]:
# setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}.")

model = LightGCN(num_users, num_books, embedding_dim=64, num_layers=2)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
# TO DO: See how useful this tuning is
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

model = model.to(device)
edge_index = edge_index.to(device)
train_edge_index = train_edge_index.to(device)
train_sparse_edge_index = train_sparse_edge_index.to(device)

val_edge_index = val_edge_index.to(device)
val_sparse_edge_index = val_sparse_edge_index.to(device)

Using device cuda.


In [ ]:
# training loop
train_losses = []
val_losses = []

for iter in range(ITERATIONS):
    # forward propagation
    # users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(train_sparse_edge_index)

    # mini batching
    user_indices, pos_item_indices, neg_item_indices = sample_mini_batch(BATCH_SIZE, train_edge_index)


    batch_sparse_edge_index = SparseTensor(
                                row=user_indices,
                                col=pos_item_indices,
                                sparse_sizes=(num_users + num_books, num_users + num_books)
                            )

    user_indices.to(device)
    pos_item_indices.to(device)
    neg_item_indices.to(device)
    batch_sparse_edge_index.to(device)

    users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(batch_sparse_edge_index)


    users_emb_final = users_emb_final[user_indices]
    users_emb_0 = users_emb_0[user_indices]
    pos_items_emb_final = items_emb_final[pos_item_indices]
    pos_items_emb_0 = items_emb_0[pos_item_indices]
    neg_items_emb_final = items_emb_final[neg_item_indices]
    neg_items_emb_0 = items_emb_0[neg_item_indices]

    # loss computation
    train_loss = bpr_loss( users_emb_final,
                           users_emb_0,
                           pos_items_emb_final,
                           pos_items_emb_0,
                           neg_items_emb_final,
                           neg_items_emb_0,
                           LAMBDA )

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if iter % ITERS_PER_EVAL == 0:
        model.eval()
        val_loss, recall, precision, ndcg = evaluation(model, val_edge_index, val_sparse_edge_index, [train_edge_index], K, LAMBDA)
        print(f"[Iteration {iter}/{ITERATIONS}] train_loss: {round(train_loss.item(), 5)}, \
                val_loss: {round(val_loss, 5)}, val_recall@{K}: {round(recall, 5)}, \
                val_precision@{K}: {round(precision, 5)}, val_ndcg@{K}: {round(ndcg, 5)}")
        train_losses.append(train_loss.item())
        val_losses.append(val_loss)
        model.train()

    # if iter % ITERS_PER_LR_DECAY == 0 and iter != 0:
    #     scheduler.step()

[Iteration 0/3500] train_loss: -0.69316,                 val_loss: -0.69281, val_recall@20: 0.00055,                 val_precision@20: 5e-05, val_ndcg@20: 0.00024
[Iteration 200/3500] train_loss: -0.69775,                 val_loss: -0.69312, val_recall@20: 0.00212,                 val_precision@20: 0.00014, val_ndcg@20: 0.00084
[Iteration 400/3500] train_loss: -0.70412,                 val_loss: -0.69366, val_recall@20: 0.03256,                 val_precision@20: 0.00192, val_ndcg@20: 0.02244
[Iteration 600/3500] train_loss: -0.71655,                 val_loss: -0.69519, val_recall@20: 0.07005,                 val_precision@20: 0.00442, val_ndcg@20: 0.05646
[Iteration 800/3500] train_loss: -0.73668,                 val_loss: -0.69824, val_recall@20: 0.09059,                 val_precision@20: 0.00604, val_ndcg@20: 0.06919
[Iteration 1000/3500] train_loss: -0.77458,                 val_loss: -0.70572, val_recall@20: 0.10604,                 val_precision@20: 0.00737, val_ndcg@20: 0.07681
[

In [ ]:
# evaluate on test set
model.eval()
test_edge_index = test_edge_index.to(device)
test_sparse_edge_index = test_sparse_edge_index.to(device)

test_loss, test_recall, test_precision, test_ndcg = evaluation(
            model, test_edge_index, test_sparse_edge_index, [train_edge_index, val_edge_index], K, LAMBDA)

print(f"[test_loss: {round(test_loss, 5)}, test_recall@{K}: {round(test_recall, 5)}, test_precision@{K}: {round(test_precision, 5)}, test_ndcg@{K}: {round(test_ndcg, 5)}")

[test_loss: -2.17727, test_recall@20: 0.13353, test_precision@20: 0.00998, test_ndcg@20: 0.09368
